## **ALgoritma Genetika**

Tahapan Algoritma:
1. Generate Populasi
2. Kalkulasi Nilai Fitness
3. Seleksi Orangtua/Parent
4. Crossover
5. Mutasi
6. Survivor Selection/Evolusi


## **Studi Kasus:**
Bagaimana membuat model penjadwalan beberapa mata kuliah ke dalam slot waktu dan ruang kelas yang tersedia, dengan mempertimbangkan:



*   Tidak ada dosen yang mengajar di dua tempat bersamaan
*  Tidak ada kelas yang dipakai dua mata kuliah di waktu sama
* Mahasiswa tidak bentrok jadwalnya







In [1]:
# import library
import random

In [2]:
# Data master
mata_kuliah = ["IF101", "IF102", "IF103", "IF104", "IF105"]
dosen = ["Anwar", "Budi", "Chika"]
hari = ["Senin", "Selasa", "Rabu"]
jam = ["08.00", "10.00", "13.00"]
ruang = ["R1", "R2"]

###**1. Generate Populasi**

In [3]:
# Representasi satu gen (satu mata kuliah)
def generate_gene(matkul):
    return {
        "matkul": matkul,
        "dosen": random.choice(dosen),
        "hari": random.choice(hari),
        "jam": random.choice(jam),
        "ruang": random.choice(ruang)
    }

# Representasi Kromosom
def generate_chromosome():
    return [generate_gene(mk) for mk in mata_kuliah]

In [4]:
#tampilkan contoh gen, misal untuk makul IF101
gene=generate_gene("IF101")
gene

{'matkul': 'IF101',
 'dosen': 'Budi',
 'hari': 'Selasa',
 'jam': '08.00',
 'ruang': 'R1'}

In [5]:
#tampilkan contoh kromosom
chromosome=generate_chromosome()
chromosome

[{'matkul': 'IF101',
  'dosen': 'Budi',
  'hari': 'Senin',
  'jam': '13.00',
  'ruang': 'R1'},
 {'matkul': 'IF102',
  'dosen': 'Budi',
  'hari': 'Rabu',
  'jam': '13.00',
  'ruang': 'R2'},
 {'matkul': 'IF103',
  'dosen': 'Budi',
  'hari': 'Selasa',
  'jam': '10.00',
  'ruang': 'R2'},
 {'matkul': 'IF104',
  'dosen': 'Budi',
  'hari': 'Senin',
  'jam': '10.00',
  'ruang': 'R1'},
 {'matkul': 'IF105',
  'dosen': 'Budi',
  'hari': 'Senin',
  'jam': '13.00',
  'ruang': 'R1'}]

### **2. Hitung Nilai Fitness**

In [6]:
# Fungsi fitness
# munggunakan fungsi minimasi
# Fungsi Objectif di tentukan dengan dengan banyaknya konflik
def calculate_fitness(chromosome):
    conflicts = 0
    for i in range(len(chromosome)):
        for j in range(i + 1, len(chromosome)):
            a, b = chromosome[i], chromosome[j]
            # Jika waktu dan hari sama:
            same_time = a["hari"] == b["hari"] and a["jam"] == b["jam"]
            if same_time:
                # Dosen mengajar di dua tempat
                if a["dosen"] == b["dosen"]:
                    conflicts += 1
                # Ruang dipakai dua kelas
                if a["ruang"] == b["ruang"]:
                    conflicts += 1
   ## return -conflicts  # makin besar (maksimal = 0), makin baik
    fitness= 1/(1+conflicts)
    return fitness


### **3. Seleksi Orangtua**

Proses seleksi orang tua disini menggunakan tournament selection

In [7]:
# Seleksi turnamen
def tournament_selection(population, k=3):
    selected = random.sample(population, k)
    return max(selected, key=lambda x: calculate_fitness(x))

### **4. Cross Over**
Cross over dilakukan dengan teknik 1 point crossover, dimana posisi titik potong ditentukan secara random/acak
misal **probabilitas cross over**= **0.8**

In [8]:
#  1 point cross over
def crossover(parent1, parent2, crossoverrate=0.8):
    if random.random() < crossoverrate:
        # posisi titik potong cross over di lakukan secara random
        point = random.randint(1, len(parent1)-1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
    else:
        # Jika tidak memenuhi probabilitas cross over /Tidak ada crossover
        # child = copy orang tua
        child1 = parent1
        child2 = parent2
    return child1, child2


### **5. Mutasi**
Misal Probabililitas mutasi=0.2

In [9]:
# Mutasi

def mutate(chromosome, mutation_rate=0.2):
    for kelas in chromosome:
        if random.random() < mutation_rate:
            kelas["jam"] = random.choice(jam)
            kelas["ruang"] = random.choice(ruang)
            kelas["hari"] = random.choice(hari)
    return chromosome

### **6. Evolusi/ Survivor Selection**
Proses evolusi dilakukan dengan konsep generational replacement, yakni mengganti seluruh populasi lama dengan populasi baru hasil reproduksi (Crossover & Mutasi)

In [10]:
# Evolusi populasi
def genetic_algorithm(generations=100, pop_size=50):
    population = [generate_chromosome() for _ in range(pop_size)]

    for gen in range(generations):
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)

            child1, child2 = crossover(parent1, parent2)
            #generational replacement
            new_population.extend([mutate(child1), mutate(child2)])
        population = new_population

        # Cek apakah sudah ada solusi tanpa konflik
        best = max(population, key=lambda x: calculate_fitness(x))
        if calculate_fitness(best) == 1:
            print(f"✅ Solusi optimal ditemukan di generasi {gen+1}!")
            return best

    # Jika belum ada solusi optimal
    best = max(population, key=lambda x: calculate_fitness(x))
    print("⚠️ Solusi optimal tidak ditemukan. Ini solusi terbaik:")
    print("Fitness:", calculate_fitness(best))
    return best

In [11]:
solusi = genetic_algorithm()
for kelas in solusi:
    print(kelas)
    print("Fitness: ",calculate_fitness(solusi))

✅ Solusi optimal ditemukan di generasi 1!
{'matkul': 'IF101', 'dosen': 'Budi', 'hari': 'Selasa', 'jam': '10.00', 'ruang': 'R1'}
Fitness:  1.0
{'matkul': 'IF102', 'dosen': 'Anwar', 'hari': 'Rabu', 'jam': '10.00', 'ruang': 'R1'}
Fitness:  1.0
{'matkul': 'IF103', 'dosen': 'Anwar', 'hari': 'Rabu', 'jam': '13.00', 'ruang': 'R2'}
Fitness:  1.0
{'matkul': 'IF104', 'dosen': 'Budi', 'hari': 'Rabu', 'jam': '10.00', 'ruang': 'R2'}
Fitness:  1.0
{'matkul': 'IF105', 'dosen': 'Budi', 'hari': 'Senin', 'jam': '08.00', 'ruang': 'R2'}
Fitness:  1.0
